In [15]:
import requests
import gzip
import pandas as pd
from io import BytesIO
import os
import shutil
from requests.exceptions import RequestException
import warnings

pd.set_option('display.max_rows', 50)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [16]:
url = "https://static.openfoodfacts.org/data/openfoodfacts-products.jsonl.gz"
fileNbr = '01'
projectPath = "/home/carolus/Documents/school/green_ia/" 
jsonGz = projectPath + "data/" + fileNbr + "_openfoodfacts" + ".jsonl.gz"
linesPerFile = 10000 # nombre de ligne pour chaque petit csv
csvPath = projectPath + "data/" + fileNbr + "_openfoodfacts_00/" 

colToSave = ['allergens_from_ingredients',
            'nutriscore_tags',
            'labels_old',
            'categories_old',
            'pnns_groups_1',
            'ecoscore_data',
            'brand_owner_imported',
            'ingredients_tags',
            'packaging',
            'ingredients_hierarchy',
            'product_name',
            'food_groups_tags',
            'ecoscore_tags',
            'nova_group',
            'ingredients_from_or_that_may_be_from_palm_oil_n',
            'categories_tags',
            'brand_owner',
            'nutrient_levels_tags',
            'allergens_tags',
            'ecoscore_extended_data',
            'categories',
            'nutriments',
            'nutriscore_2021_tags',
            'additives_old_n',
            'ecoscore_score',
            'labels_tags',
            'countries']

In [17]:
# fonction pour reprendre le téléchargement
def downloadFile(url, jsonGz):
    while True:
        try:
            # vérifier si fichier existe déjà et obtenir sa taille
            fileSize = 0
            if os.path.exists(jsonGz):
                fileSize = os.path.getsize(jsonGz)

            headers = {"range": f"bytes={fileSize}-"}
            response = requests.get(url, headers=headers, stream=True)

            if response.status_code in [200, 206]:
                mode = 'ab' if fileSize else 'wb'
                with open(jsonGz, mode) as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                print(f"fichier téléchargé et sauvegardé ici: {jsonGz}")
                break  # sortir boucle une fois téléchargement terminé

            else:
                print(f"erreur: {response.status_code}")
                break  # sortir boucle si erreur statut

        except RequestException as e:
            print(f"pause, reprise du téléchargement : {e}")

In [18]:
# décompresser du fichier jsonl
def unGzFile(jsonGz, fileNbr, projectPath):
    jsonl = projectPath + "data/" + fileNbr + '_openfoodfacts.jsonl'
    with gzip.open(jsonGz, 'rb') as f_in:
        with open(jsonl, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    print(f'fichier {jsonl} décompressé avec succès')
    return jsonl

In [19]:
# conversion en fichier csv
def convertToCsv(jsonl, fileNbr, projectPath):
    heavyCsv = projectPath + "data/" + fileNbr + '_openfoodfacts_00.csv'
    chunksize = 10000  
    chunkIter = pd.read_json(jsonl, lines=True, chunksize=chunksize)

    for i, chunk in enumerate(chunkIter):
        if i == 0:
            chunk.to_csv(heavyCsv, index=False, escapechar='\\')
        else:
            chunk.to_csv(heavyCsv, mode='a', header=False, index=False, escapechar='\\')

    print("conversion vers csv terminée")
    return heavyCsv

In [20]:
def deleteFile(filePath):
    if os.path.exists(filePath):
        os.remove(filePath)
        print(f"fichier {filePath} supprimé")
    else:
        print(f"erreur, fichier {filePath} n'existe pas")


In [21]:
def createFolder(folderPath):
    try:
        os.makedirs(folderPath, exist_ok=True)
        print(f"dossier créé avec succès: {folderPath}")
    except OSError as e:
        print(f"erreur création du dossier: {e}")

In [22]:
def splitCsv(csvFile, linesPerFile, csvPath):
    try:
        chunkSize = linesPerFile
        chunks = pd.read_csv(csvFile, chunksize=chunkSize, on_bad_lines='skip')
        
        all_columns = set()
        for chunk in chunks:
            all_columns.update(chunk.columns)
        
        chunks = pd.read_csv(csvFile, chunksize=chunkSize, on_bad_lines='skip')
        
        for i, chunk in enumerate(chunks):
            for col in all_columns:
                if col not in chunk.columns:
                    chunk[col] = None
            chunk = chunk[list(all_columns)]
            
            outputFile = f"{csvPath}{i+1}_openfoodfacts_00.csv"
            chunk.to_csv(outputFile, index=False)
            print(f"fichier {outputFile} sauvegardé avec {len(chunk)} lignes")
            
    except Exception as e:
        print(f"warning")

In [23]:
# compter les fichiers csv dans le dossier traité 
def countCsv(directory):
    csvNbr = 0
    for csvFile in os.listdir(directory):
        if csvFile.endswith('.csv'):
            csvNbr += 1
    return csvNbr

In [24]:
# lister noms de colonnes et les sauvegarder dans un fichier texte
def findAndSaveCol(df, csvPath, currentCsv):
    colName = df.columns.tolist()
    print(f"colName: {colName}")

    colTextFile = csvPath + "colSaver.txt"
    with open(colTextFile, 'w') as file:
        for name in colName:
            file.write(name + " | ")

In [25]:
# main algo

downloadFile(url, jsonGz)
jsonl = unGzFile(jsonGz, fileNbr, projectPath)
deleteFile(jsonGz)
heavyCsv = convertToCsv(jsonl, fileNbr, projectPath)
deleteFile(jsonl)
createFolder(csvPath) 
splitCsv(heavyCsv, linesPerFile, csvPath)


# compte le nombre de csv dans dossier 00
csvNbr = countCsv(csvPath)
print(f'nombre csv répertoire: {csvNbr}, fileNbr: {fileNbr}')

csvIterator = 1
while csvIterator < csvNbr:
    # initialise df
    df_00, df_01 = pd.DataFrame(), pd.DataFrame()

    currentCsv = f"{csvPath}{csvIterator}_openfoodfacts_00.csv"
    print(f"current csv: {currentCsv}")

    # traitement ici 
    df_00 = pd.read_csv(currentCsv)

    # lister toutes les colonnes du premier csv et les sauvegarder dans un fichier texte
    if csvIterator == 1:
        findAndSaveCol(df_00, csvPath, currentCsv)

    # garde dans df uniquement les colonnes utiles
    df_01 = df_00[colToSave]

    # générer un fichier csv bis avec les colonnes utiles uniquement
    df_01.to_csv(f"{csvPath}{csvIterator}_openfoodfacts_01.csv", index=False)

    # supprimer le fichier csv initial 
    deleteFile(currentCsv)

    csvIterator+=1

# compte le nombre de csv dans dossier 01
csvNbr = countCsv(csvPath)
print(f'nombre csv répertoire: {csvNbr}, fileNbr: {fileNbr}')


pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1082298688 bytes read, 6627888591 more expected)', IncompleteRead(1082298688 bytes read, 6627888591 more expected))
pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1082229134 bytes read, 5545663873 more expected)', IncompleteRead(1082229134 bytes read, 5545663873 more expected))
pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1083678222 bytes read, 4461986049 more expected)', IncompleteRead(1083678222 bytes read, 4461986049 more expected))
pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1081856926 bytes read, 3380136817 more expected)', IncompleteRead(1081856926 bytes read, 3380136817 more expected))
pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1076998446 bytes read, 2303143393 more expected)', IncompleteRead(1076998446 bytes read, 2303143393 more expected))
pause, reprise du téléchargement : ('Connection broken: Incomplet